In [6]:
import xmltodict
import csv
import pandas as pd

# PARSE XML FILE
with open('tests/data/hpo_notes.xliff') as fd:
    doc = xmltodict.parse(fd.read())


# CREATE CSV FILE
csvfile = open("tests/data/parsed_data.csv",'w',encoding='utf-8')
csvfile_writer = csv.writer(csvfile)

# ADD HEADER
csvfile_writer.writerow(['source_language', 'translation_language', 'subject_id', 'predicate_id', 'source_value','translation_value'])


for trans_unit in doc['xliff']['file']['body']['trans-unit']:

    source_language = trans_unit['source']['@xml:lang']
    
    translation_language = trans_unit['target']['@xml:lang']
    
    #get subject_id and and format
    split_string_id_column = trans_unit['@id']
    get_subject_id = split_string_id_column[0 : 10]
    subject_id = get_subject_id.replace("_", ":")

    
    #get predicate_id and format
    get_predicate_id = split_string_id_column[11 : 21]
    if get_predicate_id == 'label':
        predicate_id=get_predicate_id.replace('label', 'rdfs:label')
    elif get_predicate_id == 'definition':
        predicate_id=get_predicate_id.replace('definition', 'IAO:0000115')
    elif get_predicate_id == 'synonyms':
        predicate_id=get_predicate_id.replace('synonyms', 'oboInOwl:exactSynonym')

    
    source_value = trans_unit['source']['#text']

    translational_value = trans_unit['target']['#text']

           
    csv_line = [source_language, translation_language, subject_id, predicate_id,  source_value, translational_value]

    
    csvfile_writer.writerow(csv_line)

    
#Separate synonyms into different rows

df = pd.read_csv("tests/data/parsed_data.csv")

output_df = df[0:0]

for index, row in df.iterrows():
        if row.predicate_id == 'oboInOwl:exactSynonym':
            word_list = row['source_value'].split('#')
            word_list.remove('')
            temp_row = row.copy()
            if len(word_list) > 1:
                for each_word in word_list:
                    temp_row.source_value = each_word
                    temp_row.translation_value = each_word
                    output_df= output_df.append(temp_row.to_dict(), ignore_index = True)

            else:
                temp_row.source_value = word_list[0]
                temp_row.translation_value = temp_row.translation_value.replace('#', '')
                output_df= output_df.append(temp_row.to_dict(), ignore_index = True)
        
        else:
             output_df= output_df.append(row.to_dict(), ignore_index = True)
        
            
output_df.to_csv("tests/data/parsed_data.csv", index=False)

    
    
#     doc['xliff']['file']['body']['trans-unit']